
Lời giải cơ bản cho cuộc thi
Đây là hướng dẫn một phương pháp cơ bản dùng để phát hiện hướng di chuyển (MOI) của phương tiện giao thông trong vùng trong vùng quan sát (ROI) cùng thời điểm phương tiện rời khỏi ROI.

Hướng dẫn này sử dụng các mã nguồn mở từ các nguồn sau:

TensorFlow Object Detection API

Simple online and realtime tracking

Tổng quan phương pháp cơ bản được trình bày như sau:

Phát hiện phương tiện giao thông trong từng frame của video: Sử dụng TensorFlow Object Detection API. Kết quả trả về là một danh sách các bounding box ứng với tất cả các phương tiện giao thông trong ảnh.
Theo vết (multiple objects tracking): Dựa vào IOU (chỉ số đo đạc mức độ trùng lắp của hai bounding box), các bounding box ở các frame liên tiếp sẽ được gom nhóm và từ đó sẽ hình thành quỹ đạo di chuyển của chính phương tiện đó.
Xác định hướng di chuyển (MOI) dựa trên quỹ đạo: MOI của phương tiện sẽ được lựa chọn dựa trên độ tương đồng (ở đây sử dụng Cosine Similarity Score) giữa quỹ đạo của mỗi phương tiện và các MOI sẽ được tính toán.
Lưu ý: Trong lời giải cơ bản này, chỉ minh họa việc phát hiện và đếm phương tiện xe máy. Bạn cần chỉnh sửa và cải tiến để có thể đếm được tất cả phương tiện giao thông mà đề bài yêu cầu.

#Imports and Setup

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 30kB/s 
     |████████████████████████████████| 460kB 59kB/s 
     |████████████████████████████████| 3.0MB 36.3MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


##Cấu hình thư mục lưu trữ dữ liệu

In [ ]:
# Mount "My Drive" into /content/drive
from google.colab import drive

#google_drive_dir = 'Shared/HCMCAIC'  # @param

drive.mount('/content/drive')

#mount_point = '/content/drive/My Drive/{}'.format(google_drive_dir)

# Change the root directory to your mount_point
#% cd '$mount_point'

Mounted at /content/drive


In [ ]:
import os
import pathlib

# Clone the baseline repository 
if not pathlib.Path('ai-challenge-baseline').exists():
  ! git clone https://github.com/hcmcaic/ai-challenge-baseline

# 
% cd ai-challenge-baseline


# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'ai-challenge-baseline'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 40 (delta 9), reused 26 (delta 7), pack-reused 11
Unpacking objects: 100% (40/40), done.
/content/ai-challenge-baseline
Cloning into 'models'...
remote: Enumerating objects: 2301, done.
remote: Counting objects: 100% (2301/2301), done.
remote: Compressing objects: 100% (1993/1993), done.
remote: Total 2301 (delta 563), reused 949 (delta 285), pack-reused 0
Receiving objects: 100% (2301/2301), 30.57 MiB | 31.91 MiB/s, done.
Resolving deltas: 100% (563/563), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/ai-challenge-baseline/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1598322 sha256=320651326b8f185035035b44b580e62c9984798f94eef722c003b65040437429
  Stored in directory: /tmp/pip-ephem-wheel-cache-tzm59msk/wheels/2e/a7/0f/dae5f4550c548021aa2a057c6f045005a043a0898b4353b392
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=e594ee254e9f10878df252a600d3441bd04d27c6be79b494f9e691b7c091a1f6
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=89391015ffb3262133bdc94cc14262c176a1f8f3bcd8e6477b70e1c6bf1e0e92
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=94afb9c3d399f0212bb7bd7f25e904

ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.25.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [ ]:
# Install the requirements package for SORT source code
! pip install filterpy scikit-image lap

     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 1.5MB 8.7MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110450 sha256=4131a95b08b4e558c606fd51e708a6f7aa2f86949470416a9619b01ef239006a
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
  Created wheel for lap: filename=lap-0.4.0-cp36-cp36m-linux_x86_64.whl size=1589021 sha256=d1ea4aac6334020be47c991b91bca583c2b7e1ceb8e8beac00029a7fb55fbcb8
  Stored in directory: /root/.cache/pip/wheels/da/3e/af/eddcd6ffaa27df8d0ddac573758f8953c4e57c64c4c8c8b7d0
Successfully built filterpy lap


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from time import time
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import csv
import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.builders import model_builder


class Detector(object):
    def __init__(self, path_config, path_ckpt, path_to_labels):
        self.path_config = path_config
        self.path_ckpt = path_ckpt
        self.label_path = path_to_labels

        self.category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)
        self.detection_model = self.load_model()
        self.detection_scores = None
        self.detection_boxes = None
        self.detection_classes = None

    def detect_fn(self, image):
        """Detect objects in image."""
        image, shapes = self.detection_model.preprocess(image)
        prediction_dict = self.detection_model.predict(image, shapes)
        detections = self.detection_model.postprocess(prediction_dict, shapes)
        return detections

    def load_model(self):
        # Load pipeline config and build a detection model
        configs = config_util.get_configs_from_pipeline_file(self.path_config)
        model_config = configs['model']
        detection_model = model_builder.build(model_config=model_config, is_training=False)

        # Restore checkpoint
        ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
        ckpt.restore(self.path_ckpt).expect_partial()

        return detection_model

    def predict(self, image):
        original_img = np.copy(image)

        image = np.asarray(image)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image, 0), dtype=tf.float32)
        detections = self.detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        # num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        self.detection_scores = detections['detection_scores']
        self.detection_classes = detections['detection_classes']
        self.detection_boxes = detections['detection_boxes']

        # draw bounding boxes and labels
        image, coordinate_dict = self.draw(image)

        return image, original_img, coordinate_dict

    def draw(self, img):
        coordinate_dict = dict()
        height, width, _ = img.shape
        li = []

        for i, score in enumerate(self.detection_scores):
            self.detection_classes[i] += 1
            # if background, ignore
            # if self.detection_classes[i] == 0:
            #     continue
            # if self.detection_classes[i] != 3 and self.detection_classes[i] != 4 and self.detection_classes[i] != 6 and self.detection_classes[i] != 8:
            #     continue
            # If class is motorcycle 
            if self.detection_classes[i] == 1:
                if score < 0.3:
                  continue
            
            print("Index {} at score {} and class {} ".format(i , score, self.detection_classes[i]))                        
            label = "index " + str(i) + " class " + str(self.category_index[self.detection_classes[i]]['name']) 
            ymin, xmin, ymax, xmax = self.detection_boxes[i]
            real_xmin, real_ymin, real_xmax, real_ymax = int(xmin * width), int(ymin * height), int(xmax * width), int(
                ymax * height)

            #curr = real_xmax * real_ymax - real_ymin * real_xmin
            curr_bb = {"xmin" : real_xmin , "xmax" : real_xmax , "ymin" : real_ymin , "ymax" :real_ymax }
            status = is_overlap(curr_bb, li)
            if status == 1:
                continue
            print("Is not ovelaped")
            li.append(curr_bb)
            # check overlap bounding boxes

            cv2.rectangle(img, (real_xmin, real_ymin), (real_xmax, real_ymax), (0, 255, 0), 2)
            cv2.putText(img, label, (real_xmin, real_ymin), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
            coordinate_dict[label] = (real_xmin, real_ymin, real_xmax, real_ymax)

        return img, coordinate_dict

def get_IOU(bb1 , bb2):
    x_left = max(bb1['xmin'], bb2['xmin'])
    y_top = max(bb1['ymin'], bb2['ymin'])
    x_right = min(bb1['xmax'], bb2['xmax'])
    y_bottom = min(bb1['ymax'], bb2['ymax'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['xmax'] - bb1['xmin']) * (bb1['ymax'] - bb1['ymin'])
    bb2_area = (bb2['xmax'] - bb2['xmin']) * (bb2['ymax'] - bb2['ymin'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0
    assert iou <= 1
    return iou

def is_overlap(bb , list_bounding_box):
    for bounding_box in list_bounding_box:
        if (get_IOU(bb , bounding_box) > 0.5):
          return 1
    return 0

def check_overlap(curr, li):
    for va in li:
        # overlap
        if abs(va - curr) < 1000:
            return 1
    return 0

In [ ]:
import json
def load_zone_anno(json_filename):
  """
  Load the json with ROI and MOI annotation.

  """
  with open(json_filename) as jsonfile:
    dd = json.load(jsonfile)
    polygon = [(int(x), int(y)) for x, y in dd['shapes'][0]['points']]
    paths = {}
    for it in dd['shapes'][1:]:
      kk = str(int(it['label'][-2:]))
      paths[kk] = [(int(x), int(y)) for x, y
              in it['points']]
  return polygon, paths

In [ ]:
polygon, paths = load_zone_anno('/content/drive/My Drive/zones_movement_paths/cam_06.json')

In [ ]:
%cd '/content/ai-challenge-baseline'
from solution_baseline import bb_polygon
def check_bbox_intersect_polygon(polygon, bbox):
  """
  
  Args:
    polygon: List of points (x,y)
    bbox: A tuple (xmin, ymin, xmax, ymax)
  
  Returns:
    True if the bbox intersect the polygon
  """
  x1, y1, x2, y2 = bbox
  bb = [(x1,y1), (x2, y1), (x2,y2), (x1,y2)]
  return bb_polygon.is_bounding_box_intersect(bb, polygon)

In [ ]:
detector = Detector('/content/drive/My Drive/class1_only_v2/pipeline.config'
                    ,'/content/drive/My Drive/class1_only_v2/checkpoint/ckpt-0'
                    ,'/content/drive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt')

In [ ]:
image_dir = '/content/drive/My Drive/Frames/cam_06'

In [ ]:
from solution_baseline.sort import *
moto_tracker = Sort()

In [ ]:
import time
from google.colab.patches import cv2_imshow

In [ ]:
# Create an motobikes tracker with default parameter.
# Please read the sort documentation for the custom paramenters.
from IPython.display import clear_output
from tqdm import tqdm
import os

moto_tracker = Sort()

# If you want to track another vehicle class, you need to declare a new tracker.
# truck_tracker = Sort()


track_dict_moto = {}

N_FRAMES = 6010

#print("running")
for frame_id in tqdm(range(0, N_FRAMES ,1)):
  if frame_id % 10 == 0:
    clear_output(wait=True)
  image_path = os.path.join(image_dir, '{}.jpg'.format(frame_id))
  # print("Image path :")
  
  if not os.path.isfile(image_path):
    continue
  image_np = load_image_into_numpy_array(image_path)

  # track_image_for_bus = image_np.copy()
  # track_image_for_car = image_np.copy()
  # track_image_for_moto = image_np.copy()

  im_width, im_height, _ = image_np.shape
  input_tensor = tf.convert_to_tensor(
      np.expand_dims(image_np, 0), dtype=tf.float32)
  #detections, predictions_dict, shapes = detect_fn(input_tensor)
  #image, original_image, coordinate_dict = detector.predict(image_np)
  #cv2_imshow(image)

  # print("Visualize after get tracker")
  detections = detector.detect_fn(input_tensor)

  boxes = detections['detection_boxes'][0]
  scores = detections['detection_scores'][0]
  classes = detections['detection_classes'][0]


  dets_moto = []
  #count = -1
  for bb, s, c in zip(boxes, scores, classes):
    num_class = c + 1
    #count = count + 1
    if num_class == 1:
      if s < 0.3:
        continue  
    
    ymin, xmin, ymax, xmax = bb.numpy()
    xmin, ymin, xmax, ymax = int(xmin*im_height), int(ymin*im_width), int(xmax*im_height), int(ymax*im_width)
    if check_bbox_intersect_polygon(polygon, (xmin, ymin, xmax, ymax)):
      #print("index {} class {} Is intersect  ROI".format(count , num_class))
     
      if num_class == 1:
        dets_moto.append([xmin, ymin, xmax, ymax, s.numpy()])
    #else :
      #print("index {} class {} Is not intersect of ROI".format(count , num_class))



  dets_moto = np.array(dets_moto)


 

  if dets_moto.size != 0:
    trackers_moto = moto_tracker.update(dets_moto)
  # print('trackers')
  # print(trackers)

    for xmin, ymin, xmax, ymax, track_id in trackers_moto:
      track_id = int(track_id)
      #  Visualize tracker
      #cv2.rectangle(track_image_for_moto, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
      #cv2.putText(track_image_for_moto, "Moto Track " + str(track_id), (int(xmax), int(ymax)), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
      ##print(track_id)
      if track_id not in track_dict_moto.keys():
        track_dict_moto[track_id] = [(xmin, ymin, xmax, ymax, frame_id)]
      else:
        track_dict_moto[track_id].append((xmin, ymin, xmax, ymax, frame_id))
    # print("Trackdict")
    # print(track_dict)
    #print("Track number in moto: {}".format(len(track_dict_moto.keys())))
    #cv2_imshow(track_image_for_moto) 


#print("Finished")

In [ ]:
# moto_vector_list: list of tuples (first_point, last_point, last_frame_id)
# list of moto movement vector and the last frame_id when it is still in the ROI.

moto_vector_list = []
for tracker_id, tracker_list in track_dict_moto.items():
  # print(track_id)
  if len(tracker_list) > 1:
    first = tracker_list[0]
    last = tracker_list[-1]
    first_point = ((first[2] - first[0])/2, (first[3] - first[1])/2)
    last_point = ((last[2] - last[0])/2, (last[3] - last[1])/2)
    moto_vector_list.append((first_point, last_point, last[4]))
    a = [first_point, last_point, last[4]]
    # print(a)
    
# print(moto_vector_list)  
# print(type(moto_vector_list))

In [ ]:
def cosin_similarity(a2d, b2d):
  
  a = np.array((a2d[1][0] - a2d[0][0], a2d[1][1 ]- a2d[0][1]))
  b = np.array((b2d[1][0] - b2d[0][1], b2d[1][1] - b2d[1][0]))
  return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [ ]:
MOTO_CLASS_ID = 1

In [ ]:
def counting_moi(paths, moto_vector_list):
  """
  Args:
    paths: List of MOI - (first_point, last_point)
    moto_vector_list: List of tuples (first_point, last_point, last_frame_id) 
  
  Returns:
    A list of tuples (frame_id, movement_id, vehicle_class_id)
  """
  moi_detection_list = []
  for moto_vector in moto_vector_list:
    max_cosin = -2
    movement_id = ''
    last_frame = 0
    for movement_label, movement_vector in paths.items():
      cosin = cosin_similarity(movement_vector, moto_vector)
      if cosin > max_cosin:
        max_cosin = cosin
        movement_id = movement_label
        last_frame = moto_vector[2]

    moi_detection_list.append((last_frame, movement_id, MOTO_CLASS_ID))
  return moi_detection_list


In [ ]:
moto_moi_detections = counting_moi(paths, moto_vector_list)

In [ ]:
result_filename = 'cam_06.txt'
video_id = 'cam_06'
with open(result_filename, 'w') as result_file:
  for frame_id, movement_id, vehicle_class_id in moto_moi_detections:
    result_file.write('{} {} {} {}\n'.format(video_id, frame_id, movement_id, vehicle_class_id))